## Initial imports and global variable setup

In [1]:
%matplotlib inline
import xml.etree.ElementTree as ET
import utils
import re
from utils import *
from keras.datasets import imdb
import numpy as np
import os
from pymongo import MongoClient
import json
from translate import translator
from langdetect import detect
from random import randrange
from ipywidgets import FloatProgress
from IPython.display import display
import time
import tqdm
import dill
from numbers import Number
from collections import Counter
from operator import itemgetter
import csv
from IPython.display import SVG
from random import shuffle

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [2]:
## db variables
host = "mongodb://localhost:27017"
uid = ""
pwd = ""
client = None
db = None
sentiment_collection = None

# connect to mongo
#
print("############# initializing Mongo DB Client for infomedia articles")
mclient = MongoClient(host)
mdb = mclient.sentiments_db
infomedia_collection = mdb.infomedia

############# initializing Mongo DB Client for infomedia articles


In [3]:
## Global scope variables
GLOBAL_WORDS_LIST = []
WORD_TO_IDX = {}
IDX_TO_WORD = {}
CURRENT_LANGUAGE = "da"
current_dir = os.getcwd()
missing_words_count = 0
missing_most_common_words = []
DATA_DIR = current_dir + "/sentiments/data"
DANISH_LN_WORDS = DATA_DIR+"/danish_words.csv"
TEST_DATA_FILE_CSV = DATA_DIR+"/batch2.csv"
ORIGINAL_DATA = DATA_DIR + "/original"
TRAIN_DATA_DIR = DATA_DIR + "/train"
TEST_DATA_FILE = DATA_DIR + "/" +CURRENT_LANGUAGE+"_tests_data.p"
TEST_LABEL_FILE = DATA_DIR + "/" +CURRENT_LANGUAGE+"_tests_labels.p"
ORIGINAL_TESTS_DATA_FILE = DATA_DIR + "/" +CURRENT_LANGUAGE+"_test_results_data.p"
ORIGINAL_DATA_FILE = DATA_DIR + "/" +CURRENT_LANGUAGE+"_results_data.p"
TRAIN_DATA_FILE = DATA_DIR + "/" +CURRENT_LANGUAGE+"_train_data.p"
TRAIN_LABEL_FILE = DATA_DIR + "/" +CURRENT_LANGUAGE+"_train_labels.p"
VALIDATE_DATA_FILE = DATA_DIR + "/" +CURRENT_LANGUAGE+"_validate_data.p"
VALIDATE_LABEL_FILE = DATA_DIR + "/" +CURRENT_LANGUAGE+"_validate_labels.p"
MODEL_PATH = current_dir + "/sentiments/models/"
GLOBAL_LIST_DATA_FILE = DATA_DIR + "/"+CURRENT_LANGUAGE+"_global_list.p"
WORD_TO_IDX_DATA_FILE = DATA_DIR + "/"+CURRENT_LANGUAGE+"_words_to_idx.p"
IDX_TO_WORD_DATA_FILE = DATA_DIR + "/"+CURRENT_LANGUAGE+"_idx_to_words.p"
MISSING_WORDS_DATA_FILE = DATA_DIR + "/"+CURRENT_LANGUAGE+"_missing_words.p"
print DATA_DIR, ORIGINAL_DATA, TRAIN_DATA_DIR
# maximum num of words the sentiment can process
MAX_WORD_COUNT = 500
vocabsize = 40000
vocabsize_limit = 40000
RESERVED_PROCESS_TERM_IDX = vocabsize
DB_RECORD_EXTRACTION_LIMIT = 40000
MODEL_NAME = "model-ln-"+CURRENT_LANGUAGE+"-s"+str(vocabsize)+"-v2017-07-26"+".h5"
print MODEL_NAME

/home/ubuntu/nbs/sentiments/data /home/ubuntu/nbs/sentiments/data/original /home/ubuntu/nbs/sentiments/data/train
model-ln-da-s40000-v2017-07-26.h5


## Steps for data setup:
1. Read the articles with process terms and its sentiments.
     1.a) the output will be an array of dict items
          [{'id':'4s23s32', 'process_terms':['bestbuy, best buy'], text:'bestbuy is better than staples', 'sentiment':0.1},
           {'id':'4s23s32', 'process_terms':['staples'], text:'bestbuy is better than staples', 'sentiment':1}]
     
2. enumerate the process terms and split it into different records


In [ ]:
def load_training_data_from_db(start=0, limit_size=10):
    results = []
    total = mdb.infomedia.find({}).count()
    other_ln_count = 0
    print("******** Total records in db = "+str(total))
    cursor = mdb.infomedia.find({}).skip(0).limit(DB_RECORD_EXTRACTION_LIMIT)
#     f = FloatProgress(min=0, max=total)
#     display(f)
    for doc in cursor:
        # print(doc)
#         f.value += 1
        if not doc.get('BodyText') or doc.get('BodyText').strip() == "":
            continue
        
        r = {'id': str(doc.get('ArticleId')),
             'text': doc.get('BodyText').lower().strip(),
             'process_term': [],
             'sentiment':0.0
             }
#         print(r)
        source_ln = "da"
#         try:
#             source_ln = detect(r['text'])
#         except:
#              print("Failed to detect lang for text = "+r['text'][:30])
            
        if source_ln == CURRENT_LANGUAGE:
            process_terms= doc.get('kw')
            scores = doc.get('score')
            if process_terms != None:
                for index,item in enumerate(process_terms):
                    rcopy = r.copy()
                    for x in item:
                        pitem = x.encode("utf-8").lower()
                        rcopy['process_term'].extend([y for y in pitem.split(',')])
                    rcopy['process_term'] = set(rcopy['process_term'])   
                    rcopy['sentiment'] = scores[index]
                    results.append(rcopy)
        else:
            other_ln_count += 1
        
    
#     print("found db entries = "+results.length())
#     print results[:10]
    print("******** Total records extracted from db = "+str(len(results)))
    print("***** saving results started...please wait.....")    
    dill.dump(results, open(ORIGINAL_DATA_FILE, "wb" ))
    print("***** completed saving results ")    
    

#load_training_data_from_db();

In [ ]:
def load_test_data_from_file():
    results = []
    
    with open(TEST_DATA_FILE_CSV, 'r') as f:
        csvfile = csv.reader(f)
        skip_header = True
        pro = FloatProgress(min=0, max=50000)
        display(pro)
        pro.value = 0
        for row in csvfile:
            pro.value +=1
            if skip_header:
                skip_header = False
                continue
            r = {'process_term':[row[0]],
                          'sentiment':float(row[1]),
                          'text':row[3]
                        }
            results.append(r)
#             process_terms= r['process_term']
            
#             if process_terms != None:
#                 for index,item in enumerate(process_terms):
#                     rcopy = r.copy()
#                     for x in item:
#                         pitem = x.encode("utf-8").lower()
#                         rcopy['process_term'].extend([y for y in pitem.split(',')])
#                     rcopy['process_term'] = set(rcopy['process_term'])   
#                     results.append(rcopy)
        
        f.close()

    print("saving the test data...recs = "+str(len(results)))
    dill.dump(results, open(ORIGINAL_TESTS_DATA_FILE, "wb" ))
    print("finished!")


#load_test_data_from_file();

In [4]:
def hasNumbers(txt):
    return bool(re.search(r'\d', txt))

In [5]:
def convert_words_to_idx(text, process_terms):
    global WORD_TO_IDX
    global vocabsize
    global missing_words_count
#     global missing_most_common_words
    word_idx = []
    p_terms = [x.lower().replace('*','') for x in process_terms]
    text = text.lower().strip()
    text = text.replace("."," . ")
    text = text.replace(","," , ")
  
    for pt in p_terms:
        try:
            text = text.replace(pt, str(vocabsize + 1))
        except UnicodeDecodeError:            
            tx = text.encode('utf-8')
            text = tx.replace(pt, str(vocabsize + 1))
  

    for word in text.split():
        word = word.strip()
#         if word in p_terms:
#             word_idx.append(vocabsize + 1)
        if word == str(vocabsize + 1):
            word_idx.append(vocabsize + 1)
        else:
            if (word not in WORD_TO_IDX):
                if not hasNumbers(word):         
                    word_idx.append(vocabsize)
#                     if word not in missing_most_common_words:
#                         missing_most_common_words.append(word)
                    
#                 print ("word not found in vocab = "+word)
            else:
                word_idx.append(WORD_TO_IDX[word])
        
    
    return word_idx        

In [ ]:
def clean_words():
    global GLOBAL_WORDS_LIST
    GLOBAL_WORDS_LIST = set(pickle.load(open( GLOBAL_LIST_DATA_FILE, "rb" )))
    clean_list = [w for w in GLOBAL_WORDS_LIST if not hasNumbers(w)]
    print ("list size before cleaning = "+str(len(GLOBAL_WORDS_LIST)))
    print ("list size after cleaning = "+str(len(clean_list)))
    print("#### saving dictionary.... ")
    dill.dump(clean_list, open(GLOBAL_LIST_DATA_FILE, "wb" ))
    print("#### completed ")
#clean_words()

In [6]:
def load_words_index():
    global GLOBAL_WORDS_LIST
    global WORD_TO_IDX
    global IDX_TO_WORD
    global vocabsize
    global vocabsize_limit
    
    ## 
    # Build the word to idx and idx to word and dump it for later use
    #
    GLOBAL_WORDS_LIST = pickle.load(open( GLOBAL_LIST_DATA_FILE, "rb" ))
    
#     words_occurence = Counter(GLOBAL_WORDS_LIST)
#     print("***** total unique words = "+str(len(words_occurence)))

#     GLOBAL_WORDS_LIST = [w for w,c in words_occurence.most_common(vocabsize_limit)]
#     print ("sorted dictionary word list = ")    
    print(GLOBAL_WORDS_LIST[:20])        
    vocabsize = min(vocabsize_limit,len(GLOBAL_WORDS_LIST))
    GLOBAL_WORDS_LIST = GLOBAL_WORDS_LIST[:vocabsize]
    
    print("***** total words in dictionary = "+str(len(GLOBAL_WORDS_LIST)))
#     words_occurence = Counter(GLOBAL_WORDS_LIST)
#     print("***** total unique words = "+str(len(words_occurence)))
#     sorted_words = [w for w,c in words_occurence.most_common(vocabsize-2)]
#     print ("sorted word = ")
#     print(sorted_words[:10])
#     missing_words = pickle.load(open(MISSING_WORDS_DATA_FILE, "rb" ))
#     print("@@@ adding missing words")
#     GLOBAL_WORDS_LIST.append([w for w in missing_words if w not in GLOBAL_WORDS_LIST])
    
    WORD_TO_IDX = {w:idx for idx,w in enumerate(GLOBAL_WORDS_LIST)}
    IDX_TO_WORD = {idx:w for idx,w in enumerate(GLOBAL_WORDS_LIST)}
    assert len(WORD_TO_IDX) == len(IDX_TO_WORD)
    print ("word to idx size = "+str(len(WORD_TO_IDX)))        
    print("id to word size = "+str(len(IDX_TO_WORD)))

In [ ]:
def build_dictionary():
    global GLOBAL_WORDS_LIST
    global vocabsize
    GLOBAL_WORDS_LIST = []
    vocabsize = 0
    
    #
    # load from Danish dictionary words
    #
    print("about to open file = "+DANISH_LN_WORDS)    
    with open(DANISH_LN_WORDS, 'r') as f:
        csvfile = csv.reader(f)        
        for row in csvfile:
            if row[0] not in GLOBAL_WORDS_LIST:
                GLOBAL_WORDS_LIST.append(row[0])        
        
        f.close()
    
    ##
    # Build a word list from all the training data
    #    
    results = pickle.load(open(ORIGINAL_DATA_FILE, "rb" ))
    print("%%% about to build global word list from data files = "+str(len(results)))    
    f = FloatProgress(min=0, max=len(results))
    display(f)        
    for rec in results:
        f.value += 1
        text= rec['text'].lower()    
        text = text.replace("."," . ")
        text = text.replace(","," , ")
        text = text.replace("'"," ' ")
        text = text.replace("\""," \" ")
        words = text.split()
        GLOBAL_WORDS_LIST.extend([x.strip() for x in words if not hasNumbers(x)]) 
#         GLOBAL_WORDS_LIST.extend([x for x in words])        
    
   
        
    if "," not in GLOBAL_WORDS_LIST:
        GLOBAL_WORDS_LIST.append(",")
    if "." not in GLOBAL_WORDS_LIST:
        GLOBAL_WORDS_LIST.append(".")              
    
    
    words_occurence = Counter(GLOBAL_WORDS_LIST)
    print("***** total unique words = "+str(len(words_occurence)))

    GLOBAL_WORDS_LIST = [w for w,c in words_occurence.most_common()]
    print ("sorted dictionary word list = ")
    print(GLOBAL_WORDS_LIST[:20])
    
    vocabsize = len(GLOBAL_WORDS_LIST)
    print("#### total dictionary size = "+str(len(GLOBAL_WORDS_LIST)))
    print("#### saving dictionary.... ")
    dill.dump(GLOBAL_WORDS_LIST, open(GLOBAL_LIST_DATA_FILE, "wb" ))
    print("#### completed ")
        
#build_dictionary()
        

In [ ]:
# def build_global_words_list():
#     global GLOBAL_WORDS_LIST
#     GLOBAL_WORDS_LIST = []
#     print("%%% about to build global word list")    
#     f = FloatProgress(min=0, max=len(results))
#     display(f)
#     results = pickle.load(open(ORIGINAL_DATA_FILE, "rb" ))
#     ##
#     # Build a word list from all the training data
#     #
#     for rec in results:
#         f.value += 1
#         build_ln_word_idx_list(rec['text'])
        

In [ ]:
def setup_training_data():
    global GLOBAL_WORDS_LIST
    global WORD_TO_IDX
    global IDX_TO_WORD
    global vocabsize
    results = pickle.load(open(ORIGINAL_DATA_FILE, "rb" ))
    test_results = pickle.load(open(ORIGINAL_TESTS_DATA_FILE, "rb" ))
    
    train_data = []
    train_labels = []
    validate_data = []
    validate_labels = []
    test_data = []
    test_labels = []
    train_process_terms = [[]]
    word_list = []

             
    #
    # load the words index
    #        
    load_words_index()
    
    #    
    # convert the train_data words to indexes
    #
    print("*** converting texts to ids")
    f = FloatProgress(min=0, max=len(results))
    display(f)
    f.value = 0        
#     orderedResults = sorted(results, key=itemgetter('sentiment'))         
    for rec in results:
        f.value += 1
        src_data = (convert_words_to_idx(rec['text'].lower(),rec['process_term']))        
        src_label = (rec['sentiment'])
        
        # save every third record as a validation record
        if f.value % 3 == 0:
            validate_data.append(src_data)        
            validate_labels.append(src_label)             
        
        train_data.append(src_data)        
        train_labels.append(src_label)
            
            
    #    
    # convert test results
    #
    print("*** converting test data texts to ids")
    f.value = 0        
#     orderedTestResults = sorted(test_results, key=itemgetter('sentiment'))         
    for rec in test_results:
        f.value += 1
        test_data.append(convert_words_to_idx(rec['text'].lower(),rec['process_term']))        
        test_labels.append(rec['sentiment'])
        

#     GLOBAL_WORDS_LIST.append([w for w in missing_most_common_words if w not in GLOBAL_WORDS_LIST])
    #
    # pick random items from list for validation set
    #
#     validate_data = []
#     validate_labels = []
#     val_set_size = int(round((len(train_data) * 0.3), -1))
#     print("validation set size = "+str(val_set_size))
#     for i in range(0,val_set_size):
#         random_index = randrange(i,len(train_data))
#         validate_data.append(train_data[random_index])
#         validate_labels.append(train_labels[random_index])
#         del train_data[random_index]
#         del train_labels[random_index]
       
#     print(orderedResults[1])
    print(train_data[1])
    print(train_labels[1])
    print(validate_data[1])
    print(validate_labels[1])
    print("*** vocabsize = "+str(vocabsize))
    print("validation data size = "+str(len(validate_data)))
    print("validation labels size = "+str(len(validate_labels)))
    print("train data size = "+str(len(train_data)))
    print("train labels size = "+str(len(train_labels)))
    print("===========================================================")
    print(test_data[1])
    print(test_labels[1])
    print("test data size = "+str(len(test_data)))
    print("test labels size = "+str(len(test_labels)))
    #
    # save it for later use. 
    #
        
    print("***** saving files started...please wait.....")       
    dill.dump(train_data, open(TRAIN_DATA_FILE, "wb" ))
    print("***** saved train_data to pickle files..")
    dill.dump(train_labels, open(TRAIN_LABEL_FILE, "wb" ))
    print("***** saved train_labels to pickle files..")
    dill.dump(validate_data, open(VALIDATE_DATA_FILE, "wb" ))
    print("***** saved validate_data to pickle files..")
    dill.dump(validate_labels, open(VALIDATE_LABEL_FILE, "wb" ))
    print("***** saved validate_labels to pickle files..")  
    
    print("***** saved test_data to pickle files..")
    dill.dump(test_data, open(TEST_DATA_FILE, "wb" ))
    print("***** saved test_data to pickle files..")
    dill.dump(test_labels, open(TEST_LABEL_FILE, "wb" ))
    print("***** saved test_labels to pickle files..")  
    
    print("***** SAVED ALL FILES.....")
   


setup_training_data()


In [7]:
def load_training_data_from_file():    
    global WORD_TO_IDX
    global IDX_TO_WORD
    global vocabsize    
    
    WORD_TO_IDX = {}
    IDX_TO_WORD = {}
    
    #
    # load training data from file
    #
    train_data = pickle.load(open(TRAIN_DATA_FILE, "rb" ))
    train_labels = pickle.load(open(TRAIN_LABEL_FILE, "rb" ))    
    validate_data = pickle.load(open(VALIDATE_DATA_FILE, "rb" ))
    validate_labels = pickle.load(open(VALIDATE_LABEL_FILE, "rb" ))
    test_data = pickle.load(open(TEST_DATA_FILE, "rb" ))
    test_labels = pickle.load(open(TEST_LABEL_FILE, "rb" ))
     #
    # load the words index
    #
    load_words_index()
    
    return train_data,train_labels, validate_data, validate_labels, test_data, test_labels

## Training Code
1. load the traing and validation data from disk
2. Pad the data to match the shape of the Cnv
3. Prepare the model

In [8]:
def get_padded_data(word_idx):        
    return sequence.pad_sequences(word_idx,maxlen=MAX_WORD_COUNT, value=0)

In [9]:
t_data, t_labels, v_data, v_labels, test_data, test_labels = load_training_data_from_file()
print("===================================================================================")
print("vocab size = "+str(vocabsize))
print WORD_TO_IDX[u'konkurrence']
print IDX_TO_WORD[WORD_TO_IDX[u'konkurrence']]
print len(WORD_TO_IDX)
index = randrange(0,len(t_data))
print("===================================================================================")
print(t_data[index:index+1])
print("-------------------------------")
print(t_labels[index:index+1])
print("-------------------------------")
print("validation data size = "+str(len(v_data)))
print("validation labels size = "+str(len(v_labels)))
print("train data size = "+str(len(t_data)))
print("train labels size = "+str(len(t_labels)))
print("test data size = "+str(len(test_data)))
print("test labels size = "+str(len(test_labels)))
print Counter(test_labels)

# print(pt[index:index+1])

[u',', u'.', u'i', 'og', 'at', 'er', 'det', 'en', 'til', u'p\xe3\xa5', 'for', 'af', u'"', 'der', 'med', 'har', 'de', 'den', 'som', 'ikke']
***** total words in dictionary = 40000
word to idx size = 40000
id to word size = 40000
vocab size = 40000
846
konkurrence
40000
[[30309, 40000, 5, 20, 11, 16, 115, 1082, 14, 23677, 9, 2114, 3467, 0, 13, 9072, 11, 40000, 3, 2114, 14767, 40000, 9, 493, 1, 40000, 5, 20, 7370, 8, 16, 2704, 4890, 201, 0, 3, 16, 39384, 1034, 30, 729, 18, 20, 2350, 8, 17, 903, 0, 6278, 573, 9, 4971, 1, 27, 31, 67, 271, 10, 4, 634, 96, 9, 11603, 23, 455, 1, 0, 41, 16, 2, 217, 3861, 20869, 31, 40, 40000, 14, 4, 233, 277, 8, 7450, 3, 40000, 3295, 16, 31, 33, 74, 2089, 9, 7, 118, 11, 16, 198, 13058, 3517, 520, 3836, 612, 67, 1, 13, 31, 40, 7, 8, 65, 9, 7024, 665, 4624, 1, 3194, 9, 16, 1891, 31, 5300, 7, 18921, 3194, 14, 40000, 12, 40000, 214, 40000, 1, 16, 1891, 15, 66, 382, 7, 40000, 41, 16, 279, 1, 104, 1, 15, 4496, 18540, 0, 40000, 0, 3899, 2, 41, 174, 97, 9, 35795, 0, 3,

In [ ]:
print Counter(test_labels)

In [ ]:
print Counter(t_labels)
print Counter(v_labels)
print IDX_TO_WORD[5001]

In [10]:
pad_train_data = get_padded_data(t_data)
pad_validation_data = get_padded_data(v_data)
pad_test_data = get_padded_data(test_data)
print len(pad_train_data)
print len(pad_validation_data)
print len(pad_test_data)
print pad_train_data.shape
print pad_validation_data.shape
v_bin_labels = [1 if x>=0 else 0 for x in v_labels]
t_bin_labels = [1 if x>=0 else 0 for x in t_labels]

# cat_one_hot_encode = {-1:[1,0,0,0,0],
#                      -0.5:[0,1,0,0,0],
#                      0:[0,0,1,0,0],
#                      0.5:[0,0,0,1,0],
#                      1:[0,0,0,0,1]}
cat_one_hot_encode = {-1:[1,0,0],
                     -0.5:[1,0,0],
                     0:[0,1,0],
                     0.5:[0,0,1],
                     1:[0,0,1]}

v_cat_labels = [cat_one_hot_encode[x] for x in v_labels]
t_cat_labels = [cat_one_hot_encode[x] for x in t_labels]
test_cat_labels = [cat_one_hot_encode[x] for x in test_labels]

print v_bin_labels[:20]
print t_bin_labels[:20]
print v_cat_labels[:20]
print t_cat_labels[:20]
print test_cat_labels[:20]
print vocabsize
print MAX_WORD_COUNT
print pad_train_data[10]


76104
25368
55512
(76104, 500)
(25368, 500)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[[0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0]]
[[0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0]]
[[0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0]]
40000
500
[    0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0    

In [16]:
print test_labels[:50]

[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, -1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocabsize+2, output_dim=32, input_length=MAX_WORD_COUNT, dropout=0.2))
model.add(Dropout(0.2))
model.add(Convolution1D(64, 5, activation='relu', border_mode='same'))
model.add(Dropout(0.5))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss="binary_crossentropy", optimizer=Adam(),  metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(pad_train_data,t_labels,validation_data=(pad_validation_data, v_labels), nb_epoch=5, batch_size=20)

In [11]:
print vocabsize+1
cat_model = Sequential()
cat_model.add(Embedding(input_dim=vocabsize+2, output_dim=32, input_length=MAX_WORD_COUNT, dropout=0.2))
cat_model.add(Dropout(0.3))
cat_model.add(Convolution1D(64, 3, activation='relu', border_mode='same'))
cat_model.add(Dropout(0.2))
cat_model.add(MaxPooling1D())
cat_model.add(Flatten())
cat_model.add(Dense(100, activation='relu'))
# cat_model.add(Dropout(0.1))
cat_model.add(Dense(3, activation='softmax'))
cat_model.compile(loss="categorical_crossentropy", optimizer=Adam(),  metrics=['accuracy'])
cat_model.summary()

40001
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 500, 32)       1280064     embedding_input_1[0][0]          
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 500, 32)       0           embedding_1[0][0]                
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)  (None, 500, 64)       6208        dropout_1[0][0]                  
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 500, 64)       0           convolution1d_1[0][0]            
_____________________________________________________________________________________

In [ ]:
SVG(model_to_dot(cat_model).create(prog='dot', format='svg'))

In [17]:
print pad_train_data.shape
# print [x for x in pad_validation_data if max(x) > vocabsize+1]
#cat_model.fit(pad_train_data,t_cat_labels,validation_data=(pad_validation_data, v_cat_labels), nb_epoch=10, batch_size=64)
# cat_model.fit(pad_train_data,t_cat_labels,validation_data=(pad_test_data, test_cat_labels), nb_epoch=20, batch_size=64)
cat_model.fit(pad_test_data,test_cat_labels,validation_data=(pad_train_data, t_cat_labels), nb_epoch=5, batch_size=128)

(76104, 500)
Train on 55512 samples, validate on 76104 samples
Epoch 1/5
55512/55512 [==============================] - 38s - loss: 0.5950 - acc: 0.7458 - val_loss: 0.4264 - val_acc: 0.8209
Epoch 2/5
55512/55512 [==============================] - 40s - loss: 0.4302 - acc: 0.8241 - val_loss: 0.4791 - val_acc: 0.7891
Epoch 3/5
55512/55512 [==============================] - 38s - loss: 0.3533 - acc: 0.8577 - val_loss: 0.4942 - val_acc: 0.7911
Epoch 4/5
55512/55512 [==============================] - 38s - loss: 0.3005 - acc: 0.8822 - val_loss: 0.5786 - val_acc: 0.7660
Epoch 5/5
55512/55512 [==============================] - 40s - loss: 0.2597 - acc: 0.8972 - val_loss: 0.4832 - val_acc: 0.8030


In [ ]:
#binary model
model.save_weights(MODEL_PATH + MODEL_NAME)

In [18]:
#categorical modeling
cat_model.save_weights(MODEL_PATH + "ty_cat_"+MODEL_NAME)

## Test / Predict

In [ ]:
model.load_weights(MODEL_PATH + MODEL_NAME)

In [19]:
cat_model.load_weights(MODEL_PATH + "ty_cat_"+MODEL_NAME)

In [37]:
# text="The board network Board Network, which mentions itself as \"Denmark\'s most exclusive board network\", has presented a financial statement for 2016, which is filled with errors and remarks by the auditor.Board Network is a company that makes networking for board members and organizes several events for its members. A membership is paid in NOK 10,000 per year before VAT.Board Networks advisory committee has known members such as Jens Moberg, Chairman of the Board of PostNord and Grundfos, and Eivind Kolding, former director of Danske Bank Prominent management center is in deep crisis. But the accounts of the celebre club get a lot of hug from the auditor, who does not make a conclusion on the accounts at all.Firstly, equity is negative, and the auditor has not seen an acceptable plan for how to change it.In addition, there are a series of legitimate offenses that may end with punishment to the owner.'The company is in violation of the Companies Act section 210, paragraph 1. 1, granted a loan to the company\'s capital owner, whereby management can incur liability. ''In violation of the Withholding Tax Act, the company has not contained and reported taxes thereon, whereby the management may incur liability.''The company has in breach of the VAT Act reported incorrect VAT returns to Tax, which may lead to liability.'\"In violation of the Danish Financial Statements Act, the company has not prepared the annual report in a timely manner, whereby the management can incur liability, the auditor strikes.\"\"We can never be happy to make an account with remarks.\"He emphasizes that there is no risk of the bankruptcy of the company.According to the financial statements, the company has granted a loan to Jacob Stengel, which is in violation of the Companies Act."
# text = "Hasbro-holdet leverede endnu et stærkt kvartal på tværs af vores brands."
# text = "which mentions itself as \"Denmark\'s most exclusive board"
#text = "Bestyrelsesnetværket Board Network, der nævner sig som \"Danmarks mest eksklusive bestyrelsesnetværk \", har aflagt en finansieringsoversigt for 2016, som er fyldt med fejl og bemærkninger fra revisor. Board Network er et firma, der gør netværk Til bestyrelsesmedlemmer og organiserer flere arrangementer for sine medlemmer. Et medlemskab er betalt i kr. 10.000 pr. År før moms. Børnetværkets rådgivende udvalg har kendt medlemmer som Jens Moberg, bestyrelsesformand for PostNord og Grundfos, og Eivind Kolding, tidligere direktør for Danske Bank Prominent Management Center er i dyb krise. Men celebre-klubbernes regnskaber får en masse kram fra revisoren, som ikke har en konklusion på regnskabet. For det første er egenkapitalen negativ, og revisoren har ikke set en acceptabel plan for, hvordan man ændrer det. Derudover er der en række lovlige lovovertrædelser, der kan ende med straf for ejeren. «Selskabet overtræder aktielovens § 210, stk. 1. 1, ydet et lån til selskabets kapitalindehaver, hvorved ledelsen Kan pådrage sig ansvar. '' I strid med skatteloven har selskabet ikke indeholdt og rapporteret skatter deraf, hvorved ledelsen kan pådrage sig ansvar. '' Selskabet har i strid med momsloven rapporteret ukorrekt momsafkast til skat, hvilket kan medføre ansvar . '\ "I strid med årsregnskabsloven har selskabet ikke udarbejdet årsrapporten rettidigt, hvorved ledelsen kan pådrage sig ansvar, revisor slår. \" \ "Vi kan aldrig være glade for at lave en konto Med bemærkninger. "Han understreger, at der ikke er nogen risiko for selskabets konkurs. Ifølge årsregnskabet har selskabet ydet lån til Jacob Stengel, der er i strid med selskabsloven."
#text = "Bestyrelsesnetværket Board Network, der nævner sig som \"Danmarks mest eksklusive bestyrelsesnetværk \", har aflagt en finansieringsoversigt for 2016, som er fyldt med fejl og bemærkninger fra revisor."
text = "Bestyrelsesnetværket Board Network, For det første er egenkapitalen negativ, og revisor har ikke set en acceptabel plan for, hvordan man ændrer den. Derudover er der en række lovlige lovovertrædelser, der kan ende med straf til ejeren"
# text = "19 personer er blevet syge af en type salmonella-bakterie, der spores tilbage til en færdigret med forloren hare fra Coop."
processed_terms = [
        "Board Network"
    ]

In [35]:
category_keys = ["-1   (Very Negative)","-0.5 (Negative)","0    (Neutral)" ,"0.5  (Positive)","1    (Very Positive)"]

def predict(text, process_terms):
    
    global category_keys
    
    textWordsIdxs = convert_words_to_idx(text,processed_terms)
    textIdxArrayPadded = get_padded_data([np.array(textWordsIdxs)])
    
    print cat_model.predict(textIdxArrayPadded, batch_size=1,verbose=1)
    
    print textIdxArrayPadded
#     preds = dict(zip(category_keys, cat_model.predict(textIdxArrayPadded, batch_size=1,verbose=1)[0]))
#     ordered_preds_keys = sorted(preds,key=preds.get, reverse=True)    
#     num = float(((ordered_preds_keys[0])[:5]).strip())
    
#     return num
#     print ("\n\n------------------- model prediction (probabilities) -------------------")
#     print("\n".join([x.ljust(20)+" \t\t "+str(preds[x]) for x in ordered_preds_keys]))
    # print("\n".join(ordered_preds_keys))
#     print("\n\n====================R E S U L T=====================================")`
#     print "sentiment = "+ordered_preds_keys[0]+" = "+str(preds[ordered_preds_keys[0]])
    

In [36]:
print predict(text,processed_terms)

1/1 [==============================] - 0s
[[ 0.0089  0.9671  0.024 ]]
[[    0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     

In [24]:
#text_clean = re.sub('\W+', ' ', text)
print len(text.split())
category_keys = ["-1   (Very Negative)","-0.5 (Negative)","0    (Neutral)" ,"0.5  (Positive)","1    (Very Positive)"]
textWordsIdxs = convert_words_to_idx(text,processed_terms)
textIdxArrayPadded = get_padded_data([np.array(textWordsIdxs)])
print textIdxArrayPadded
preds = dict(zip(category_keys, cat_model.predict(textIdxArrayPadded, batch_size=1,verbose=1)[0]))
ordered_preds_keys = sorted(preds,key=preds.get, reverse=True)
print ("\n\n------------------- model prediction (probabilities) -------------------")
print("\n".join([x.ljust(20)+" \t\t "+str(preds[x]) for x in ordered_preds_keys]))
# print("\n".join(ordered_preds_keys))
print("\n\n====================R E S U L T=====================================")
print "sentiment = "+ordered_preds_keys[0]+" = "+str(preds[ordered_preds_keys[0]])
# print ordered_preds
# print "probability difference = "+str(preds[ordered_preds[0]] - preds[ordered_preds[1]])
# if (preds[ordered_preds[0]] - preds[ordered_preds[1]]) > 0.10:

# else:
#     print "sentiment = "+str(ordered_preds[1])
# for i,x in enumerate(prediction[0]):
#     if x == max(prediction[0]):
#         print "sentiment = "+str(categories[i])



20
[[    0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
      0     0    

In [28]:
# cat_model.test_on_batch(pad_test_data[:20000],test_cat_labels[:20000])
# scores = cat_model.evaluate(pad_test_data, test_cat_labels, batch_size=128)
scores = cat_model.evaluate(pad_test_data, test_cat_labels, batch_size=128)
print("%s: %.2f%%" % (cat_model.metrics_names[1], scores[1]*100))
print scores

scores = cat_model.evaluate(pad_train_data, t_cat_labels, batch_size=128)
print("%s: %.2f%%" % (cat_model.metrics_names[1], scores[1]*100))
print scores

55512/55512 [==============================] - 12s    
acc: 95.35%
[0.16878117737551557, 0.95350554832413692]
75904/76104 [============================>.] - ETA: 0sacc: 80.30%
[0.48318916401326262, 0.8030195521792064]
